In [ ]:
from inference.select_best_models import *
# tar cf best_models.tar best_models
# tar cf saved_models.tar saved_models

In [ ]:
gpu_num=2
target_prop="dft_e_hull"

model_params = {
    "model_type": "CGCNN",
    "relaxed": False,
    "interpolation": False,
}

# reverify_sigopt_models(model_params, gpu_num, target_prop)
# keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=2
target_prop="dft_e_hull"

model_params = {
    "model_type": "CGCNN",
    "relaxed": True,
    "interpolation": False,
}

# reverify_sigopt_models(model_params, gpu_num, target_prop)
# keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=2
target_prop="dft_e_hull"

model_params = {
    "model_type": "CGCNN",
    "relaxed": False,
    "interpolation": True,
}

# reverify_sigopt_models(model_params, gpu_num, target_prop)
# keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=2
target_prop="dft_e_hull"

model_params = {
    "model_type": "CGCNN",
    "relaxed": True,
    "interpolation": True,
}

# reverify_sigopt_models(model_params, gpu_num, target_prop)
# keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=2
target_prop="dft_e_hull"

model_params = {
    "model_type": "Painn",
    "relaxed": False,
    "interpolation": True,
}

# reverify_sigopt_models(model_params, gpu_num, target_prop)
# keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=2
target_prop="dft_e_hull"

model_params = {
    "model_type": "Painn",
    "relaxed": True,
    "interpolation": True,
}

# reverify_sigopt_models(model_params, gpu_num, target_prop)
# keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=2
target_prop="dft_e_hull"

model_params = {
    "model_type": "e3nn",
    "relaxed": False,
    "interpolation": True,
}

# reverify_sigopt_models(model_params, gpu_num, target_prop)
# keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=2
target_prop="dft_e_hull"

model_params = {
    "model_type": "e3nn",
    "relaxed": True,
    "interpolation": True,
}

# reverify_sigopt_models(model_params, gpu_num, target_prop)
# keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "model_type": "e3nn_contrastive",
    "relaxed": False,
    "interpolation": True,
}

# reverify_sigopt_models(model_params, gpu_num, target_prop)
# keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "model_type": "e3nn_contrastive",
    "relaxed": True,
    "interpolation": True,
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)